# Notebook pour réaliser l'extraction des posts et commentaires

* __Description__: Notebook pour l'extraction des commentaires et des posts de Reddit pour leur sauvegarde dans BigQuery
* __Source__: API de REDDIT
* __Output__: Tables comment et post (BigQuery) 
* __Auteur__: Corentin TIMAL et Camille MATTHIEU
* __Date de création__: 03/08/2022
* __Date de mise à jour__: 14/09/2022

### Import des outils

In [1]:
%run /notebook/Extraction_reddit/libs/extraction_lib.ipynb

## Extraction complète

Cette fonction permet d'importer dan la liste des posts et des commentaires pour un subreddit donné

In [ ]:
def extraction_complete(subreddit:str):
    """
        Fonction qui, quand on lui fournit le subreddit, va récupérer de façon automatique :
        - ses posts
        - les commentaires associés à chacun des posts
        Param:
            subreddit: str, nom du subreddit au format "nom_subreddit"
    """ 
    
    posts_extraction(subreddit)
    
    # Recup BigQuery
    date = date_extraction[:10]
    heure = date_extraction[11:]
    
    sql = f"SELECT id_post FROM `mimetic-coral-355913.dwh.post` WHERE extraction_utc = '{date}T{heure}'"
    
    # BigQuerry to Pandas DataFrame
    pandas_df = client.query(sql).to_dataframe()
    # Pandas DF to Spark DataFrame
    spark_df = spark.createDataFrame(pandas_df)
    # Spark DF to Spark RDD
    rdd = spark_df.rdd
    # flatmap pour faire qlqch
    rdd2 = rdd.map(lambda row : comments_extraction(f"r/{subreddit}", row["id_post"]))
    rdd3 = rdd2.reduce(lambda df1, df2: pd.concat([df1, df2]))
    
    insertionBigQuery('dwh', 'comment', rdd3)
    